## Data Cleaning & Validation Exercise

In [249]:
import pandas as pd
import numpy as np
import spacy
# nlp = spacy.load('en_core_web_sm')
import en_core_web_sm
nlp = en_core_web_sm.load()

pd.options.display.max_rows = 999

In [250]:
df = pd.read_csv(r'data\WELLCOME_APCspend2013_forThinkful.csv', encoding = 'unicode_escape')

In [251]:
# I'm going to want to change the cost column to int data type to be able to perform math
df.dtypes

PMID/PMCID                                             object
Publisher                                              object
Journal title                                          object
Article title                                          object
COST (£) charged to Wellcome (inc VAT when charged)    object
dtype: object

In [252]:
# remove spaces, change to lowercase
df.columns = df.columns.str.replace(' ', '_')
df.columns = map(str.lower, df.columns)

# abbreviate cost column
df.rename(columns={'cost_(£)_charged_to_wellcome_(inc_vat_when_charged)': "cost"}, inplace=True)

In [253]:
df.head()

,pmid/pmcid,publisher,journal_title,article_title,cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [254]:
df.shape

(2127, 5)

## 1. Determine the five most common journals and the total articles for each. 

In [255]:
# text clearly isn't consistent between journal articles
df.journal_title.value_counts().head()

PLoS One                                           92
PLoS ONE                                           62
Journal of Biological Chemistry                    48
Nucleic Acids Research                             21
Proceedings of the National Academy of Sciences    19
Name: journal_title, dtype: int64

In [256]:
# lowercase journal names
df.journal_title = df.journal_title.apply(lambda x: str(x).lower())

# strip whitespace
df.journal_title = df.journal_title.apply(lambda x: str(x).strip())

In [257]:
df.journal_title.value_counts().head(20)

plos one                                           190
journal of biological chemistry                     53
neuroimage                                          29
nucleic acids research                              26
plos pathogens                                      24
plos genetics                                       24
proceedings of the national academy of sciences     22
plos neglected tropical diseases                    20
nature communications                               19
human molecular genetics                            19
movement disorders                                  15
bmc public health                                   14
brain                                               14
journal of neuroscience                             13
developmental cell                                  12
biochemical journal                                 12
current biology                                     11
journal of general virology                         11
malaria jo

In [258]:
# how many journal articles contain 'plos'
df.loc[df.journal_title.str.contains('plos'), 'journal_title'].value_counts()

plos one                            190
plos genetics                        24
plos pathogens                       24
plos neglected tropical diseases     20
plosone                               9
plos computational biology            9
plos 1                                7
plos                                  4
plos medicine                         4
plos biology                          2
plos  computational biology           1
plos ntd                              1
plos  one                             1
plos negected tropical diseases       1
plos medicine journal                 1
Name: journal_title, dtype: int64

In [259]:
# create list of values to change based on similarities
plos_one = ['plos 1',
            'plos  one',
            'plosone',
            'plos']

In [260]:
# is there a better way to do this with a lambda function?
df.loc[df.journal_title.isin(plos_one), 'journal_title'] = 'plos one'

In [261]:
# now verify value counts
df.loc[df.journal_title.str.contains('plos'), 'journal_title'].value_counts()

plos one                            211
plos genetics                        24
plos pathogens                       24
plos neglected tropical diseases     20
plos computational biology            9
plos medicine                         4
plos biology                          2
plos  computational biology           1
plos ntd                              1
plos negected tropical diseases       1
plos medicine journal                 1
Name: journal_title, dtype: int64

In [262]:
# there are still a number of similar names
df.journal_title.sort_index().head(10)

0            psychological medicine
1                 biomacromolecules
2                        j med chem
3                        j med chem
4                        j org chem
5    journal of medicinal chemistry
6      journal of proteome research
7                         mol pharm
8              acs chemical biology
9              acs chemical biology
Name: journal_title, dtype: object

In [263]:
df.journal_title.value_counts().head()

plos one                           211
journal of biological chemistry     53
neuroimage                          29
nucleic acids research              26
plos pathogens                      24
Name: journal_title, dtype: int64

## 2. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal.

In [264]:
# strip the £ character off the front
df.cost = df.cost.apply(lambda x: x[1:])

# I could have also use the .str.replace('£', '') to do the same as the above

In [265]:
# remove $ character
df.cost = df.cost.str.replace('$', '')

In [266]:
# change format of column to numeric
df.cost = pd.to_numeric(df.cost)

In [267]:
# verify data types are correct
df.dtypes

pmid/pmcid        object
publisher         object
journal_title     object
article_title     object
cost             float64
dtype: object

In [268]:
df.cost.value_counts().head(10)

2040.00      94
999999.00    47
1500.00      37
2400.00      32
2100.00      30
1800.00      28
3000.00      26
1700.00      21
825.68       19
1834.77      18
Name: cost, dtype: int64

In [269]:
# remove values that seem out of proportion
replace = {
    999999.0: 0
}

df.replace({'cost': replace}, inplace=True)

In [270]:
df.groupby('journal_title')['cost'].agg(['mean', 'median', 'std']) \
  .sort_values(by='mean', ascending=False).head(10)

,mean,median,std
journal_title,,,
movement disorders,15176.788000,2010.24,51414.460186
nan,13200.000000,13200.00,NaN
public service review,6000.000000,6000.00,NaN
the lancet neurology,5040.000000,5040.00,1018.233765
the lancet,4558.003333,4554.01,240.024915
cell host & microbe,4226.040000,4226.04,NaN
curr biol.,4151.770000,4151.77,NaN
cell journal,4041.050000,4041.05,NaN
immunity,3934.750000,3934.75,190.791552


## 3. Identify the open access prices paid by subject area.

1. for the top 50 journal titles
1. identify key words that seem to go together, form them into lists
1. use those lists to assign the journals into categories
1. maybe identify 5 categories, see how many remain after that

### Method 1

In [271]:
# df.journal_title.value_counts().head(10)

In [272]:
# brain = ('cereb'or
#         'brain'or
#         'neuro'or
#          'cortex')

# # seems like I can't do a list if I want to use str.contains 
# # in order to hunt down smaller patterns in journal titles
# # is a string chained together the best option here?
# disease = ('virol'or
#           'immun'or
#           'malaria'or
#           'patho'or
#           'disea'or
#           'infec'or
#           'para')

# df.loc[df.journal_title.str.contains(brain), 'journal_bin'] = 'brain'

# df.loc[df.journal_title.str.contains(disease), 'journal_bin'] = 'disease'

In [273]:
# df.loc[df.journal_bin.notnull()].shape

### Method 2

In [274]:
# split up each journal title based on spaces
# df['split_journals'] = df.journal_title.str.split(' ')

In [275]:
# df.head(10)

In [276]:
# assign labels based on key words
# df.loc[df.journal_title.str.contains('biol'), 'journal_bin2'] = 'biology'
# df.loc[df.journal_title.str.contains('gene'), 'journal_bin2'] = 'genetics'
# df.loc[df.journal_title.str.contains('neuro'), 'journal_bin2'] = 'neurology'
# df.loc[df.journal_title.str.contains('psych'), 'journal_bin2'] = 'psychology'
# df.loc[df.journal_title.str.contains('endo'), 'journal_bin2'] = 'endocrinology'
# df.loc[df.journal_title.str.contains('epid'), 'journal_bin2'] = 'epidemiology'
# df.loc[df.journal_title.str.contains('pharma'), 'journal_bin2'] = 'pharmacology'

In [277]:
# this seemed much more effective
# df.loc[df.journal_bin2.notnull()].shape

In [278]:
# some aren't labeled quite properly
# for instance chemical biology is being assigned to biology right now

# df.loc[df.journal_bin2.notnull()].head(20)

In [279]:
# journal_words_df_count.head(90)

### Method 3

### Identify Top Journal Keywords

In [281]:
df.head()

,pmid/pmcid,publisher,journal_title,article_title,cost
0,NaN,CUP,psychological medicine,Reduced parahippocampal cortical thickness in ...,0.00
1,PMC3679557,ACS,biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04
2,23043264 PMC3506128,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56
3,23438330 PMC3646402,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,669.64
4,23438216 PMC3601604,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,685.88


In [287]:
# break up tokens from journal titles
# find the count of each token
# based on the tokens with the highest counts, form buckets

journal_words = []

for row in df.journal_title.str.split():
    for word in row:
        if len(word) > 3:
            journal_words.append(word)

journal_words_df = pd.DataFrame(journal_words, columns=['tokens'])

journal_words_df_count = journal_words_df.groupby('tokens')['tokens'] \
    .count() \
    .reset_index(name= 'token_count') \
    .sort_values(by='token_count', ascending=False)

In [289]:
journal_words_df_count.head()

,tokens,token_count
371,journal,528
539,plos,298
96,biology,92
595,research,91
423,molecular,88


In [290]:
# lemmatize and tag jornal bag of words
# https://stackoverflow.com/questions/44395656/applying-spacy-parser-to-pandas-dataframe-w-multiprocessing

lemma = []
pos = []
tag = []

for doc in nlp.pipe(journal_words_df_count['tokens'].astype('unicode').values, batch_size=50):
    lemma.append([n.lemma_ for n in doc])
    pos.append([n.pos_ for n in doc])
    tag.append([n.tag_ for n in doc])

journal_words_df_count['lemma'] = lemma
journal_words_df_count['pos'] = pos
journal_words_df_count['tag'] = tag

In [291]:
# change format of columns from list to string
# is there a better way I could be doing this?
journal_words_df_count.lemma = journal_words_df_count.lemma.apply(''.join)
journal_words_df_count.pos = journal_words_df_count.pos.apply(''.join)

In [292]:
journal_words_df_count.head()

,tokens,token_count,lemma,pos,tag
371,journal,528,journal,NOUN,[NN]
539,plos,298,plo,NOUN,[NNS]
96,biology,92,biology,NOUN,[NN]
595,research,91,research,NOUN,[NN]
423,molecular,88,molecular,ADJ,[JJ]


In [293]:
# lemmatize journal titles
lemma = []

for doc in nlp.pipe(df.journal_title.astype('unicode').values, batch_size=50):
    lemma.append([n.lemma_ for n in doc])
#     pos.append([n.pos_ for n in doc])

df['journal_lemma'] = lemma

In [294]:
df.head()

,pmid/pmcid,publisher,journal_title,article_title,cost,journal_lemma
0,NaN,CUP,psychological medicine,Reduced parahippocampal cortical thickness in ...,0.00,"[psychological, medicine]"
1,PMC3679557,ACS,biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04,[biomacromolecule]
2,23043264 PMC3506128,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56,"[j, med, chem]"
3,23438330 PMC3646402,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,669.64,"[j, med, chem]"
4,23438216 PMC3601604,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,685.88,"[j, org, chem]"


## Results Comparison/Contrast

### Using spaCy Lemmatization
I felt that spaCy didn't find a very deep root of the word.  Resulting in less effective binning

In [295]:
# idenfity only nouns
# exclude the top 2 since they are not meaningful
journal_words_df_count.loc[journal_words_df_count.pos == 'NOUN', 'lemma'][2:52].head()

96       biology
595     research
148    chemistry
135         cell
292      genetic
Name: lemma, dtype: object

In [296]:
# iterate through journal lemma words
# then bin based on word 
# ranked most highly in the bag of words
def bin_based_on_stem(stemmed_journal):
    for word in stemmed_journal:
        if word in journal_words_df_count.loc[journal_words_df_count.pos == 'NOUN', 'lemma'][2:52].values:
            return word
    return np.nan

In [297]:
df['lemma_bin'] = df.journal_lemma.apply(bin_based_on_stem)

In [298]:
df.head()

,pmid/pmcid,publisher,journal_title,article_title,cost,journal_lemma,lemma_bin
0,NaN,CUP,psychological medicine,Reduced parahippocampal cortical thickness in ...,0.00,"[psychological, medicine]",medicine
1,PMC3679557,ACS,biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04,[biomacromolecule],NaN
2,23043264 PMC3506128,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56,"[j, med, chem]",NaN
3,23438330 PMC3646402,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,669.64,"[j, med, chem]",NaN
4,23438216 PMC3601604,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,685.88,"[j, org, chem]",NaN


In [299]:
df.lemma_bin.value_counts().head()

chemistry    83
cell         81
genetic      80
biology      76
medicine     57
Name: lemma_bin, dtype: int64

In [300]:
# how many journals were assigned?
df.lemma_bin.value_counts().sum()

1307

In [301]:
# how many journals in total
df.shape[0]

2127

In [302]:
# what percent of journals were assigned?
df.lemma_bin.value_counts().sum() / \
df.shape[0]

0.614480488951575

In [318]:
# a glance at the journals that weren't binned with this method
df[df.lemma_bin.isna()].head(20)

,pmid/pmcid,publisher,journal_title,article_title,cost,journal_lemma,lemma_bin,first_4_char_bin
1,PMC3679557,ACS,biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04,[biomacromolecule],NaN,biomacromolecule
2,23043264 PMC3506128,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56,"[j, med, chem]",NaN,chem
3,23438330 PMC3646402,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,669.64,"[j, med, chem]",NaN,chem
4,23438216 PMC3601604,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,685.88,"[j, org, chem]",NaN,chem
7,23057412 PMC3495574,ACS,mol pharm,Quantitative silencing of EGFP reporter gene b...,649.33,"[mol, pharm]",NaN,NaN
13,PMC3413243,ACS Publications,biochemistry,Monomeric 14-3-3 has a chaperone-like activity...,665.64,[biochemistry],NaN,biochemistry
15,PMC3572711,AGA Institute,gastroenterology,Much of the genetic risk of colorectal cancer ...,238.08,[gastroenterology],NaN,NaN
23,PMCID: 3584654,AMERICAN CHEMICAL SOCIETY,acs nano,HYDROXY-TERMINATED CONJUGATED POLYMER NANOPART...,642.89,"[acs, nano]",NaN,nano
34,23373658,American Chemical Society Publications,acs nano,Skin dendritic cell targeting via microneedle ...,693.39,"[acs, nano]",NaN,nano
35,PMC3673662,American College of Chest Physicians,chest,Synthetic response of stimulated respiratory e...,2383.94,[chest],NaN,NaN


### Using First 4 Journal Name Characters
This is a bit more crude, however I felt as if it did a better job overall

In [303]:
# further refine words based on the first 4 digits
journal_words_df_count['first_4_char'] = journal_words_df_count.lemma.apply(lambda x: str(x)[0:4])

In [320]:
# groupby first 4 characters and order by largest number of tokens consolidated
journal_words_df_count.groupby('first_4_char')['first_4_char'].count().sort_values(ascending=False).head(100).index

Index(['neur', 'chem', 'tran', 'psyc', 'gene', 'immu', 'biom', 'expe', 'bioc',
       'bioe', 'inte', 'micr', 'prot', 'biol', 'soci', 'arch', 'geno', 'beha',
       'reso', 'infe', 'proc', 'stat', 'medi', 'anti', 'para', 'chil', 'crys',
       'scie', 'bios', 'phys', 'cell', 'clin', 'diab', 'deve', 'phil', 'card',
       'biop', 'resp', 'radi', 'repr', 'onco', 'circ', 'comm', 'comp', 'nano',
       'derm', 'ange', 'viro', 'meta', 'jour', 'epid', 'infl', 'ther', 'hist',
       'heal', 'haem', 'afri', 'amer', 'visu', 'phar', 'evol', 'epil', 'plan',
       'geog', 'inve', 'ethn', 'info', 'huma', 'fron', 'expr', 'cent', 'chro',
       'phot', 'oxid', 'hear', 'pedi', 'func', 'hepa', 'joun', 'cogn', 'epig',
       'diff', 'mode', 'mole', 'dise', 'econ', 'magn', 'muta', 'lond', 'neph',
       'nerv', 'leuk', 'cyto', 'nutr', 'curr', 'cult', 'endo', 'engi', 'opin',
       'opth'],
      dtype='object', name='first_4_char')

In [335]:
# iterate through first 4 characters of journal lemma words
# then bin based on word 
def bin_based_on_stem(stemmed_journal):
    for word in stemmed_journal:
        if word[0:4] in journal_words_df_count.groupby('first_4_char')['first_4_char'].count().sort_values(ascending=False).head(100).index:
            if word[0:4] != 'jour':
                return word
    return np.nan

In [336]:
df['first_4_char_bin'] = df.journal_lemma.apply(bin_based_on_stem)

In [339]:
df.head()

,pmid/pmcid,publisher,journal_title,article_title,cost,journal_lemma,lemma_bin,first_4_char_bin
0,NaN,CUP,psychological medicine,Reduced parahippocampal cortical thickness in ...,0.00,"[psychological, medicine]",medicine,psychological
1,PMC3679557,ACS,biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04,[biomacromolecule],NaN,biomacromolecule
2,23043264 PMC3506128,ACS,j med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56,"[j, med, chem]",NaN,chem
3,23438330 PMC3646402,ACS,j med chem,Orvinols with mixed kappa/mu opioid receptor a...,669.64,"[j, med, chem]",NaN,chem
4,23438216 PMC3601604,ACS,j org chem,Regioselective opening of myo-inositol orthoes...,685.88,"[j, org, chem]",NaN,chem


In [340]:
# how many journals were assigned?
df.first_4_char_bin.value_counts().sum()

1520

In [341]:
# how many journals in total
df.shape[0]

2127

In [342]:
# what percent of journals were assigned?
df.first_4_char_bin.value_counts().sum() / \
df.shape[0]

0.7146215326751293

In [343]:
# which journal titles weren't binned using this method
df[df.first_4_char_bin.isna()].head(20)

,pmid/pmcid,publisher,journal_title,article_title,cost,journal_lemma,lemma_bin,first_4_char_bin
15,PMC3572711,AGA Institute,gastroenterology,Much of the genetic risk of colorectal cancer ...,238.08,[gastroenterology],NaN,NaN
35,PMC3673662,American College of Chest Physicians,chest,Synthetic response of stimulated respiratory e...,2383.94,[chest],NaN,NaN
48,PMC3444304,American Psychological Association,emotion,Subjective responses to emotional stimuli duri...,2534.53,[emotion],NaN,NaN
92,PMC3754575,American Society for Microbiology,journal of bacteriology,The agr locus sregulates virulence and coloniz...,1343.82,"[journal, of, bacteriology]",NaN,NaN
96,PMCID:\n PMC3518918\n\n,American Society for Microbiology,mbio,EspZ of enteropathogenic and enterohemorrhagic...,1586.75,[mbio],NaN,NaN
101,NaN,American Society of Haematology,blood,Disease-associated missense mutations in the E...,1260.26,[blood],NaN,NaN
102,NaN,American Society of Haematology,blood,SAP gene transfer restores cellular and humora...,1260.26,[blood],NaN,NaN
103,23255554 PMC3653566,American Society of Hamatology,blood journal 2012,Peptide Length Determines the Outcome of T Cel...,1597.98,"[blood, journal, 2012]",NaN,NaN
104,NaN,American Society of Hematology,blood,Super-resolution imaging of remodeled synaptic...,1230.40,[blood],NaN,NaN
105,PMC3501641,American Society of Hematology,blood,Induction of IL-4Ra-dependent microRNAs identi...,1288.14,[blood],NaN,NaN
